# Cahn-Hilliard Equation

This is the implementation for Cahn-Hilliard Equation for two phase separation in 2D using FEniCSx.
The boundary-value problem is described by:
$$\frac{\partial c}{\partial t} - \nabla \cdot M \left( \nabla \left( \frac{\partial f}{\partial c} - \lambda \Delta c \right) \right) = 0 \quad \in \Omega$$

We use split formulation to rephrase the fourth-order equation into two coupled second-order equations:
$$
\begin{aligned}
\frac{\partial c}{\partial t} - \nabla \cdot M \nabla \mu &= 0 \quad \in \Omega\\
\mu - \frac{\partial f}{\partial c} + \lambda \Delta c &= 0 \quad \in \Omega
\end{aligned}
$$

## Variational formulation

### Weak formulation

The variational form of the equations are:
$$
\begin{aligned}
\int_\Omega \frac{\partial c}{\partial t} q dx + \int_\Omega M \nabla \mu \cdot \nabla q dx = 0 \\
\int_\Omega \mu v dx - \int_\Omega \frac{\partial f}{\partial c} v dx - \int_\Omega \lambda \nabla c \cdot \nabla v dx = 0
\end{aligned}
$$

### Crank-Nicholson time stepper

The sampling of the first PDE at time $t_{n+1}$ is given by:
$$
\begin{aligned}
\int_\Omega \frac{c_{n+1} - c_n}{dt} q dx + \frac{1}{2} \int_\Omega M \nabla \left( \mu_{n+1} + \mu_n \right) \cdot \nabla q dx = 0 \\
\int_\Omega \mu_{n+1} v dx - \int_\Omega \frac{d f_{n+1}}{dc} v dx - \int_\Omega \lambda \nabla c_{n+1} \cdot \nabla v dx = 0
\end{aligned}
$$

## Problem definition

- Domain: $\Omega = [0,1] \times [0,1]$
- Local energy: $f = 100 c^2 (1 - c)^2$
- Gradient energy coefficent: $\lambda = 1 \times 10^{-2}$
- Mobility coefficient: $M = 1$
- Dirichlet BCs: $u = 0$ on $\{(0,y) \cup (2,y)\} \in \partial\Omega$

In [2]:
# Import Libraries
from mpi4py import MPI
from dolfinx import mesh, fem, plot, io, log, default_real_type
import numpy as np
import ufl
from basix.ufl import element, mixed_element
from dolfinx.fem.petsc import NonlinearProblem
from dolfinx.nls.petsc import NewtonSolver
from petsc4py import PETSc
from pathlib import Path

In [3]:
# Define simulation parameters
lambda_ = 1.0e-2
dt = 5.0e-06
T = 1.0e-03 # End time
num_steps = T / dt # Number of time steps

# Create mesh
domain = mesh.create_unit_square(MPI.COMM_WORLD, 100, 100, cell_type=mesh.CellType.triangle)

# Create FunctionSpace
P1 = element("Lagrange", domain.basix_cell(), 1, dtype=default_real_type)
ME = fem.functionspace(domain, mixed_element([P1, P1]))

# Apply Periodic BCs
# Identify the facets on the left and right boundaries
facets = mesh.locate_entities_boundary(domain, domain.topology.dim - 1, lambda x: np.full(x.shape[1], True, dtype=bool))

# Define variational problem
# Define trial and test functions
u = fem.Function(ME)
# Previous solution
u0 = fem.Function(ME)
q, v = ufl.TestFunctions(ME)
# Split mixed functions
c, mu = ufl.split(u)
c0, mu0 = ufl.split(u0)

# Initial condition
u.x.array[:] = 0.0
rng = np.random.default_rng(42)
u.sub(0).interpolate(lambda x: 0.63 + 0.02 * (0.5 - rng.random(x.shape[1])))
u.x.scatter_forward()

# Compute df/dc
c = ufl.variable(c)
f = 100 * c**2 * (1 - c) ** 2
dfdc = ufl.diff(f, c)

# Define residuals
F0 = ufl.inner(c, q) * ufl.dx - ufl.inner(c0, q) * ufl.dx + (dt/2) * ufl.inner(ufl.grad(mu + mu0), ufl.grad(q)) * ufl.dx
F1 = ufl.inner(mu, v) * ufl.dx - ufl.inner(dfdc, v) * ufl.dx - lambda_ * ufl.inner(ufl.grad(c), ufl.grad(v)) * ufl.dx
F = F0 + F1

# Create NonlinearProblem
problem = NonlinearProblem(F, u)

# Create Newton Solver
log.set_log_level(log.LogLevel.INFO)
solver = NewtonSolver(domain.comm, problem)
solver.convergence_criterion = "incremental"
solver.rtol = 1.0e-6
ksp = solver.krylov_solver
opts = PETSc.Options()  # type: ignore
option_prefix = ksp.getOptionsPrefix()
opts[f"{option_prefix}ksp_type"] = "preonly"
opts[f"{option_prefix}pc_type"] = "lu"
sys = PETSc.Sys()
opts[f"{option_prefix}pc_factor_mat_solver_type"] = "superlu_dist"
ksp.setFromOptions()

# Post-process
# Save solution to file
t = 0.0
results_folder = Path("results")
results_folder.mkdir(exist_ok=True, parents=True)
filename = results_folder / "out_ch2p"
with io.XDMFFile(domain.comm, filename.with_suffix(".xdmf"), "w", io.XDMFFile.Encoding.ASCII) as xdmf:
    xdmf.write_mesh(domain)
# Get the sub-space for c
V0, dofs = ME.sub(0).collapse()

# Time-stepping
c = u.sub(0)
u0.x.array[:] = u.x.array
while t < T:
    t += dt
    r = solver.solve(u)
    print(f"Step {int(t / dt)}: num iterations: {r[0]}")
    u0.x.array[:] = u.x.array
    xdmf.write_function(c, t)
xdmf.close()

[2025-02-28 08:15:24.956] [info] Column ghost size increased from 0 to 0
[2025-02-28 08:15:24.971] [info] Adding mesh to node "/Xdmf/Domain"
[2025-02-28 08:15:24.971] [info] Adding topology data to node /Xdmf/Domain/Grid
[2025-02-28 08:15:24.973] [info] Adding geometry data to node "/Xdmf/Domain/Grid"
[2025-02-28 08:15:25.007] [info] PETSc Krylov solver starting to solve system.
[2025-02-28 08:15:26.655] [info] PETSc Krylov solver starting to solve system.
[2025-02-28 08:15:27.713] [info] Newton iteration 2: r (abs) = 0.22696988829013243 (tol = 1e-10), r (rel) = 0.00018541545143229064 (tol = 1e-06)
[2025-02-28 08:15:27.726] [info] PETSc Krylov solver starting to solve system.
Step 1: num iterations: 3[2025-02-28 08:15:28.222] [info] Newton iteration 3: r (abs) = 3.883733704305332e-06 (tol = 1e-10), r (rel) = 3.1726862248179646e-09 (tol = 1e-06)
[2025-02-28 08:15:28.222] [info] Newton solver finished in 3 iterations and 3 linear solver iterations.

[2025-02-28 08:15:28.223] [info] Addin